<a href="https://colab.research.google.com/github/SarveshD7/DCGAN-Pytorch/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# ***Important Points on the Architecture of Official Implementation of DCGAN***
The Generator and Discriminator use the same set of Blocks but just ar the opposites of each other.

The Generator uses Transpose Convolutions for UpScaling the image, whereas Discriminator uses normal Convolutions.

Official Implementation of DCGAN does not use BatchNorm in only the First Layer of Discriminaotor and Last Layer of Generator.

So We need to specifically write these layers and use _block for rest.

Since we are using BatchNorm in all other layers we do bias=False for them.

---



In [23]:
class Discriminator(nn.Module):
  def __init__(self, channels_img, features_d):  # features_d is channels that are going to change in different layers
    super(Discriminator, self).__init__()
    self.disc = nn.Sequential(
        # Input: N x channels_img x 64 x 64
        nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),  # 32x32
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d*2, 4,2,1),  # 16x16
        self._block(features_d*2, features_d*4, 4,2,1),  # 8x8
        self._block(features_d*4, features_d*8, 4,2,1),  # 4x4
        nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0), # 1x1
        nn.Sigmoid()  # Finally we get a single value telling us whether the image passed is real or fake
    )

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
        # nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2)
    )

  def forward(self,x):
    return self.disc(x)


In [24]:
class Generator(nn.Module):
  def __init__(self, z_dim, channels_img, features_g):
    super(Generator, self).__init__()
    # Input: N x z_dim x 1 x 1
    self.gen = nn.Sequential(
        self._block(z_dim, features_g*16, 4, 1, 0), # 4x4
        self._block(features_g*16, features_g*8, 4, 2, 1), # 8x8
        self._block(features_g*8, features_g*4, 4, 2, 1), # 16x16
        self._block(features_g*4, features_g*2, 4, 2, 1), # 32x32
        nn.ConvTranspose2d(features_g*2,channels_img, kernel_size=4, stride=2, padding=1),  # 64x64
        nn.Tanh()  # Range = [-1,1]
    )

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
        # nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

  def forward(self, x):
    return self.gen(x)


In [25]:
# Initializing weights of the model with a normal distribution with given mean and standard deviation
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.02)

In [26]:
def test():
  N, in_channels, H, W = 8, 3, 64, 64
  z_dim = 100
  x = torch.randn((N, in_channels, H, W))
  disc = Discriminator(in_channels, 8)
  initialize_weights(disc)
  assert disc(x).shape == (N, 1, 1, 1)
  gen = Generator(z_dim, in_channels, 8)
  initialize_weights(gen)
  z = torch.randn((N, z_dim, 1, 1))
  assert gen(z).shape == (N, in_channels, H, W)
  print("Success")

In [27]:
test()

Success


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

In [29]:
transforms = torchvision.transforms.Compose(
   [ torchvision.transforms.Resize(IMAGE_SIZE),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)],
    )]
)

In [30]:
dataset = datasets.MNIST(root="/content/dataset/", train=True, transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [31]:
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

In [32]:
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [33]:
fixed_noise = torch.randn(32, Z_DIM,1,1).to(device)

In [34]:
gen.train()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU()
    )
    (4): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): Tanh()
  )
)

In [35]:
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [36]:
# Wrting the training loop
for epoch in range(NUM_EPOCHS):
  print(f"Started epoch-> {epoch}")
  for batch_idx, (real, _) in enumerate(loader):
    real = real.to(device)
    noise = torch.randn((BATCH_SIZE, Z_DIM, 1, 1)).to(device)
    fake = gen(noise)
    # Train Discriminator--> Maximize log(D(x)) + log(1-D(G(z)))
    disc_real = disc(real).reshape(-1)
    loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
    # criterion-> BCE Loss-> ℓ(x,y) = -wn*[yn . logxn +(1-yn) . log(1-xn)] -> We are passing y as all ones so ulitmately this term will become the first component of the Discriminator Loss
    disc_fake = disc(fake).reshape(-1)
    loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    # criterion-> BCE Loss-> ℓ(x,y) = -wn*[yn . logxn +(1-yn) . log(1-xn)] -> We are passing y as all zeros so ulitmately this term will become the second component of the Discriminator Loss
    loss_disc = (loss_disc_real + loss_disc_fake)/2
    disc.zero_grad()
    loss_disc.backward(retain_graph=True)  # retain_graph=True since we want to reuse fake and Pytorch erases intermediate results on loss.backward()
    opt_disc.step()

    # Training the Generator---> Minimize log(1-D(G(z)) <----> Maximize log(D(g(z)))
    output = disc(fake).reshape(-1)
    loss_gen = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()
  print(f"Completed epoch-> {epoch}")


Started epoch-> 0


KeyboardInterrupt: ignored